In [12]:
# Import delle librerie necessarie
import json  # per leggere e scrivere file JSON

# Import della classe Tabella2D_RO per il bonus
try:
    from utilities.griglia_di_liste_RO import Tabella2D_RO
    # Inizializziamo la tabella con una riga vuota per evitare errori di costruttore
    table = Tabella2D_RO([[]])
    HAS_TAB2D = True
except Exception as e:
    table = None
    HAS_TAB2D = False
    print(f"[WARN] Tabella2D_RO non disponibile: {e}")

print("Tabella2D_RO disponibile:", HAS_TAB2D)

Tabella2D_RO disponibile: True


In [14]:
# --- PARAMETRI ---
file_input = "test_data/test_large.json"  # percorso del file di log
file_output = "output_traccia4.json"      # file JSON di output

In [16]:
# Lettura dei log dal file JSON
try:
    with open(file_input, "r", encoding="utf-8") as f:
        logs = json.load(f)  # carica i log in una lista di liste
except Exception as e:
    print(f"[ERROR] Lettura file {file_input} fallita: {e}")
    exit(1)

print(f"Totale log caricati: {len(logs)}")

Totale log caricati: 83627


In [18]:
# Dizionario finale per utente
# Struttura: 
# { "user_id": { "ip_diversi": [lista_IP], "conteggio_ip": {IP: conteggio} } }
info_utenti = {}

# Ciclo su tutti i log
for idx, log in enumerate(logs):
    # Controllo che ogni log sia una lista di 8 elementi come richiesto
    if not isinstance(log, list) or len(log) != 8:
        print(f"[WARN] Log {idx} non valido: {log}")
        continue  # salta log non validi

    user = log[1]   # Identificativo unico dell'utente (2° elemento)
    ip = log[7]     # Indirizzo IP (8° elemento)

    # Se l'utente non è ancora presente, inizializza la struttura
    if user not in info_utenti:
        info_utenti[user] = {
            "ip_diversi": set(),  # set temporaneo per IP unici
            "conteggio_ip": {}    # conteggio eventi per IP
        }

    # Aggiungi IP al set
    info_utenti[user]["ip_diversi"].add(ip)

    # Aggiorna conteggio eventi per quell'IP
    if ip not in info_utenti[user]["conteggio_ip"]:
        info_utenti[user]["conteggio_ip"][ip] = 1
    else:
        info_utenti[user]["conteggio_ip"][ip] += 1

    # --- BONUS: memorizzazione log nella Tabella2D_RO ---
    if HAS_TAB2D:
        try:
            # Append del log alla tabella read-only
            table._data4rows.append(log)
            # Ricostruzione delle colonne
            table._data4cols = [[table._data4rows[i][j] for i in range(len(table._data4rows))] 
                                for j in range(len(table._data4rows[0]))]
        except Exception as e:
            print(f"[WARN] impossibile inserire log in Tabella2D_RO: {e}")

# Convertiamo i set in liste ordinate per JSON
for user in info_utenti:
    info_utenti[user]["ip_diversi"] = sorted(list(info_utenti[user]["ip_diversi"]))

In [20]:
# Salvataggio del risultato in un file JSON leggibile
try:
    with open(file_output, "w", encoding="utf-8") as f:
        json.dump(info_utenti, f, indent=4, ensure_ascii=False)
    print(f"Elaborazione completata! File salvato come: {file_output}")
except Exception as e:
    print(f"[ERROR] Salvataggio file {file_output} fallito: {e}")

Elaborazione completata! File salvato come: output_traccia4.json


In [22]:
if HAS_TAB2D:
    print("\n--- Esempi bonus Tabella2D_RO ---")
    
    # Dimensioni della tabella
    righe, colonne = table.size()
    print(f"Dimensioni tabella: {righe} righe x {colonne} colonne")
    
    # Consultare la 10ª riga (se esiste)
    if righe > 10:
        print(f"Riga 10: {table.get_riga(9)}")
    
    # Consultare la colonna degli IP (8° colonna) se esiste
    if colonne >= 8:
        print(f"Colonna degli IP (8°): {table.get_colonna(7)[:10]}...")  # mostriamo solo primi 10


--- Esempi bonus Tabella2D_RO ---
Dimensioni tabella: 83628 righe x 0 colonne
Riga 10: ['3/11/2021 15:19', '00001', 'Corso: Fondamenti di informatica [20-21]', 'Sistema', 'Visualizzato corso', "The user with id '7' viewed the course with id '488'.", 'web', '172.17.16.141']


In [24]:
from datetime import datetime   # per convertire stringhe in oggetti datetime
from collections import defaultdict  # per creare dizionari annidati con valori di default
import statistics  # per calcolo media e varianza
import json  # per salvare i risultati in formato JSON

# --- Parametri ---
file_output_statistiche = "statistiche_settimanali.json"  # file di output per le statistiche

# --- Struttura dati per contare accessi per settimana ---
# utilizziamo defaultdict annidato: ogni utente ha un dizionario che mappa (anno, settimana) -> numero accessi
# Esempio: accessi_settimanali["utente1"][(2025, 42)] = 3
accessi_settimanali = defaultdict(lambda: defaultdict(int))

# --- Ciclo su tutti i log già caricati in logs ---
for log in logs:
    try:
        # log[0] contiene la data e ora in formato "%d/%m/%Y %H:%M"
        data_ora = datetime.strptime(log[0], "%d/%m/%Y %H:%M")
        utente = log[1]  # log[1] è l'identificativo dell'utente

        # Otteniamo anno e numero della settimana ISO (1–53)
        anno, settimana, _ = data_ora.isocalendar()

        # Incrementiamo il numero di accessi per quell'utente e quella settimana
        accessi_settimanali[utente][(anno, settimana)] += 1

    except Exception as e:
        # Se il log non è valido (es. data malformattata), stampiamo un warning e saltiamo il log
        print(f"[WARN] Impossibile processare log {log}: {e}")
        continue

# --- Calcolo delle statistiche settimanali per ogni utente ---
statistiche = {}

for utente, settimane in accessi_settimanali.items():
    # lista dei conteggi di accessi per ogni settimana
    lista_accessi = list(settimane.values())

    # calcolo della media degli accessi settimanali
    media = statistics.mean(lista_accessi)

    # calcolo della varianza; se esiste solo una settimana, la varianza = 0
    varianza = statistics.variance(lista_accessi) if len(lista_accessi) > 1 else 0

    # memorizzazione dei risultati per ogni utente
    statistiche[utente] = {
        "media_accessi_settimanali": media,
        "varianza_accessi_settimanali": varianza,
        "numero_settimane_con_accessi": len(lista_accessi)
    }

# --- Salvataggio delle statistiche in JSON ---
try:
    with open(file_output_statistiche, "w", encoding="utf-8") as f:
        # indent=4 per leggibilità, ensure_ascii=False per supportare caratteri non ASCII
        json.dump(statistiche, f, indent=4, ensure_ascii=False)
    print(f"Calcolo completato! File salvato come: {file_output_statistiche}")
except Exception as e:
    print(f"[ERROR] Salvataggio file {file_output_statistiche} fallito: {e}")

Calcolo completato! File salvato come: statistiche_settimanali.json
